## This is training bigram language model with the short frankenstein dataset

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
with open('data/raw/frankenstein.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print(text[200:400])

 evil
forebodings. I arrived here yesterday, and my first task is to assure
my dear sister of my welfare and increasing confidence in the success
of my undertaking.

I am already far north of London, 


In [4]:
chars = sorted(set(text))
vocab_size = len(chars)
learning_rate = 3e-4
max_iter = 2000
# print(chars)
# print(len(chars))

In [5]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [6]:
data = torch.tensor(encode(text), dtype=torch.long)

print(data[:100])

tensor([32, 51, 66, 66, 51, 64,  1,  9,  0,  0, 46, 39, 61,  1, 33, 64, 65,  7,
         1, 38, 47, 68, 55, 58, 58, 51,  5,  1, 25, 60, 53, 58, 47, 60, 50,  7,
        46,  0,  0,  0, 38, 66,  7,  1, 36, 51, 66, 51, 64, 65, 48, 67, 64, 53,
        54,  5,  1, 24, 51, 49,  7,  1,  9,  9, 66, 54,  5,  1,  9, 15, 78,  7,
         0,  0,  0, 43, 61, 67,  1, 69, 55, 58, 58,  1, 64, 51, 56, 61, 55, 49,
        51,  1, 66, 61,  1, 54, 51, 47, 64,  1])


In [7]:
n = int(0.8 * len(data))
train_data = data[:n]
test_data = data[n:]
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4

cuda


In [8]:
def get_batch(split):
    data = train_data if split == 'train' else test_data
    ix = torch.randint(len(data) - block_size, (batch_size,))

    # print(ix)

    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    # print("x:", x)
    # print("y:", y)

    return x, y

In [9]:
x, y = get_batch('train')

In [10]:
print(torch.cuda.device_count())

1


In [11]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'test']:
        losses = torch.zeros(1000)
        for k in range(1000):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [12]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [13]:
model = BigramLanguageModel(vocab_size=vocab_size)

m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)

generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())

print(generated_chars)


A7“‘JBèaFrCYx1è(s[Oæ“rOtHè(dKG3n,0kiR;CK2F”K2”]yn1Jh6MJ)v:LUT?fn[LU2tdu?A[)d3pObaES[xhfiF1-
plwG,_‘NPeôrcséhlæz88i03n;BVhlw:S!Vs’?u5SMpc3!h-jW7HA4W(uPeo[Vp5CnBiwjFb1aRN_mBCR2”aKAvei‘SmOeoVn!gVdqé k4lCg5qé01Oz[Dp]n_fa_i_—YWVy6YB5bxO57pH(2tIéuk,abSyd.y3èk2—PuLê’A-5-LnOqM2_‘g7GMP5èFy9y
xOsn3pjoôsq7:bYTlfê’389!h(u]fôl,èOoleC)—;GT!iP‘NLTP58qzOsqô0kU —qDUSfu
bGEôu—”2Yvb?TuqKDA[bx”uwaWCwCBôssuiEJ[Oæ2hM9pv4KlfuLUJz8a9OEn8ccéCL?i_]j(GLjJCangUa5C;2OzOvrr8xKWæD“r:
gmx08qôsmGMKd_bh6jvRuw
;Cawybxrêè5b4W1v1y6


In [14]:
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

for i in range(max_iter):
    if i % 500 == 0:
        losses = estimate_loss()
        print(f"step {i}: train loss {losses['train']:.4f}, test loss {losses['test']:.4f}")
    xb, yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

step 0: train loss 5.0014, test loss 5.0008
step 500: train loss 4.8624, test loss 4.8775
step 1000: train loss 4.7327, test loss 4.7435
step 1500: train loss 4.6089, test loss 4.6009
4.403591156005859


In [15]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)

generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())

print(generated_chars)


I7é]”a5h(,4u—fu?2OA_GTxV]M90hf-KmBYWSdpjwly”vh(IkeP1n4)mBVY—7[_mKRN‘Y)Drbur]
V.é0e4w“fô-5JCC,vuSoeEmBè’TA[YM’qY0MBV‘‘ét.S!unrdMRN:S4DG_HIW3VKK4é5[æFeY4æg!Moc[s,7gn[ks?6(hè—F”Vpf“æCMRjê_Hn—é5,nrj,;K!æ89’akè(—rEaFqS”3ês[V;l 6D[”:U_mKd3!m,xcèPLRL22)ju9.? p’KHv9ki?pY3sM2M2_kdKx‘MB.i”R;w!W
 yvH]-Lê2OnJhT,mTPx1è7tIKxBsu P:—é5h(“r:7_K;Vôs[V]yu?CwThNu.1P1-Kæ3lTanENghoATj8H48(u5TPSPOYvôA7i-C“VkUPthlô0hIKô!BeNkbAl!AvFIfonf1DIYês3Rc29WIT5CMvp]-T]AuR“c3“ukègub7N3A7é5ôswyS_ieC(IYDrdi-,j1F2ôWV]ôsLci:n,’]M CSF


In [16]:
input = torch.rand((4, 8, 10))
B, T, C = input.shape
output = input.view(B*T, C)
print(output)
print(output[:, -1])

tensor([[0.2614, 0.8201, 0.6108, 0.6099, 0.9468, 0.2027, 0.4201, 0.3709, 0.0372,
         0.7046],
        [0.9665, 0.2480, 0.5409, 0.6666, 0.4512, 0.7444, 0.9035, 0.5925, 0.6643,
         0.7800],
        [0.3299, 0.1288, 0.8896, 0.4882, 0.9520, 0.6204, 0.3664, 0.4424, 0.9795,
         0.8607],
        [0.5628, 0.9545, 0.7894, 0.2103, 0.4152, 0.5301, 0.5751, 0.4200, 0.7714,
         0.2632],
        [0.6302, 0.4421, 0.2696, 0.3380, 0.4930, 0.8043, 0.8058, 0.9885, 0.4748,
         0.0607],
        [0.2415, 0.7352, 0.2297, 0.2505, 0.6610, 0.9001, 0.9442, 0.8623, 0.1589,
         0.5244],
        [0.0620, 0.8471, 0.8327, 0.2066, 0.4711, 0.6449, 0.3586, 0.8949, 0.8456,
         0.1658],
        [0.6092, 0.8822, 0.7033, 0.9077, 0.4298, 0.2365, 0.2287, 0.0269, 0.2663,
         0.3500],
        [0.6886, 0.4005, 0.1077, 0.4031, 0.3640, 0.7288, 0.4086, 0.9565, 0.7075,
         0.0115],
        [0.9521, 0.7675, 0.5094, 0.9617, 0.7222, 0.4640, 0.5774, 0.6737, 0.0786,
         0.6971],
        [0

In [17]:
x = torch.tensor([-0.05], dtype=torch.float32)
Y = F.relu(x)
print(Y)

tensor([0.])


In [20]:
print(torch.cuda.get_device_name())

NVIDIA GeForce RTX 3060 Laptop GPU


In [23]:
Y = F.sigmoid(x)
print(Y)

tensor([0.4875])


In [25]:
Y = F.tanh(x)
print(Y)

tensor([-0.0500])
